In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 174
USE = 'carts'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
LOAD = 709
OFFSET = ''

df = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

(1801251, 2)


,session_type,labels
3602502,11098528_carts,11830 1732105 588923 1157882 884502 1182614 87...
3602503,11098529_carts,1105029 1049489 53004 1694360 628708 459126 13...
3602504,11098530_carts,409236 264500 1603001 963957 254154 364155 583...
3602505,11098531_carts,396199 1271998 452188 1728212 1365569 624163 1...
3602506,11098532_carts,876469 7651 1159379 476681 669555 1202618 1081...


In [4]:
OFFSET = '_wgt'

df2 = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

(1801251, 2)


,session_type,labels
3602502,11098528_carts,-1 36.65 35.15 29.149999999999995 28.950000000...
3602503,11098529_carts,-1 19.275 13.125 12.174999999999999 10.55 10.3...
3602504,11098530_carts,-1 -1 48.825 33.375 33.025 30.175 28.325000000...
3602505,11098531_carts,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 22.0272727272...
3602506,11098532_carts,-1 -1 25.575 25.275000000000002 23.50000000000...


In [5]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.drop('session_type',axis=1)
df.head()

,labels,user
3602502,11830 1732105 588923 1157882 884502 1182614 87...,11098528
3602503,1105029 1049489 53004 1694360 628708 459126 13...,11098529
3602504,409236 264500 1603001 963957 254154 364155 583...,11098530
3602505,396199 1271998 452188 1728212 1365569 624163 1...,11098531
3602506,876469 7651 1159379 476681 669555 1202618 1081...,11098532


In [6]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.drop('session_type',axis=1)
df2.head()

,labels,user
3602502,-1 36.65 35.15 29.149999999999995 28.950000000...,11098528
3602503,-1 19.275 13.125 12.174999999999999 10.55 10.3...,11098529
3602504,-1 -1 48.825 33.375 33.025 30.175 28.325000000...,11098530
3602505,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 22.0272727272...,11098531
3602506,-1 -1 25.575 25.275000000000002 23.50000000000...,11098532


In [7]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,item,pos
3602502,11830,0
3602502,1732105,1
3602502,588923,2
3602502,1157882,3
3602502,884502,4


In [8]:
users = df[['user']].copy()
users.head()

,user
3602502,11098528
3602503,11098529
3602504,11098530
3602505,11098531
3602506,11098532


In [9]:
users = users.merge(tmp, left_index=True, right_index=True)

In [10]:
del df,tmp
gc.collect()

43

In [11]:
users.head()

,user,item,pos
3602619,11098645,598073,44
3602619,11098645,1511252,45
3602619,11098645,661338,46
3602619,11098645,1315302,47
3602619,11098645,1278983,48


In [12]:
users.dtypes

user    int32
item    int32
pos     int32
dtype: object

In [13]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

,wgt,pos
3602502,-1.000000,0
3602502,36.650002,1
3602502,35.150002,2
3602502,29.150000,3
3602502,28.950001,4


In [14]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

,user
3602502,11098528
3602503,11098529
3602504,11098530
3602505,11098531
3602506,11098532


In [15]:
del df2
gc.collect()

22

In [16]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [17]:
users2.head()

,user,wgt,pos
3602619,11098645,2.150,44
3602619,11098645,2.125,45
3602619,11098645,2.000,46
3602619,11098645,2.000,47
3602619,11098645,2.000,48


In [18]:
users = users.merge(users2,on=['user','pos'],how='left')

In [19]:
users.head()

,user,item,pos,wgt
0,11098530,124606,88,1.0
1,11098530,876017,89,1.0
2,11098530,718069,90,1.0
3,11098530,685123,91,1.0
4,11098530,524335,92,1.0


In [20]:
del users2, tmp
gc.collect()

79

In [21]:
users.shape

(180125100, 4)

# Add Targets

In [22]:
test_labels = cudf.read_parquet('../../data/train_data/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]
print( test_labels.shape )
test_labels.head()

(306341, 3)


,session,type,ground_truth
1,11098528,carts,[1199737]
8,11098533,carts,"[108676, 1406660, 988295, 1118792, 1366413, 15..."
11,11098534,carts,[223062]
15,11098536,carts,[649909]
18,11098538,carts,"[1263747, 668757, 1550143]"


In [23]:
tmp = test_labels.ground_truth.explode().astype('int32').rename('item')
tmp.head()

1    1199737
8     108676
8    1406660
8     988295
8    1118792
Name: item, dtype: int32

In [24]:
test_users = test_labels[['session']].astype('int32').copy()
test_users.columns = ['user']

In [25]:
test_users = test_users.merge(tmp,left_index=True,right_index=True)

In [26]:
del test_labels, tmp
gc.collect()

50

In [27]:
test_users[USE] = np.int8(1)

In [28]:
test_users.dtypes

user     int32
item     int32
carts     int8
dtype: object

In [29]:
test_users.head()

,user,item,carts
4247,11101813,98676,1
4247,11101813,667733,1
4247,11101813,1524654,1
4247,11101813,1506701,1
4250,11101815,1668042,1


In [30]:
users = users.merge(test_users, on=['user','item'], how='left').fillna(0)

In [31]:
del test_users
gc.collect()

40

In [32]:
users.head()

,user,item,pos,wgt,carts
0,11098723,1812412,52,0.658074,0
1,11098723,1825691,53,0.651297,0
2,11098723,1263108,54,0.640000,0
3,11098723,510056,55,0.625000,0
4,11098723,90800,56,0.614964,0


In [33]:
print(users.shape, users.dtypes)

(180125100, 5) user       int32
item       int32
pos        int32
wgt      float32
carts       int8
dtype: object


In [34]:
users = users.to_pandas()

# Filter Rows

In [35]:
%%time
tmp = users.groupby('user')[USE].agg('sum')
print( len(tmp) )
KEEP = tmp.loc[tmp>0].index.values
print( len(KEEP), len(KEEP)/len(tmp) )
del tmp
gc.collect()

1801251
233887 0.1298469785721146
CPU times: user 3.6 s, sys: 680 ms, total: 4.28 s
Wall time: 4.28 s


29

In [36]:
users = users.loc[users.user.isin(KEEP)].reset_index(drop=True)
print( users.shape )
gc.collect()

(23388700, 5)


0

In [37]:
users[USE].mean(), 1/users[USE].mean()

(0.012984176119237068, 77.01682346394101)

# Save

In [38]:
users.dtypes

user       int32
item       int32
pos        int32
wgt      float32
carts       int8
dtype: object

In [39]:
users[USE] = users[USE].astype('int8')

In [40]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}.pqt',index=False)